In [1]:
import os

In [3]:
%pwd

'c:\\Users\\Lenovo\\Downloads\\ML Projects\\Brain-Tumor-Detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Lenovo\\Downloads\\ML Projects\\Brain-Tumor-Detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.cnnClassifier.constants import * 
from src.cnnClassifier.utils.common import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) ->DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir= config.root_dir,
                source_URL= config.source_URL,
                local_data_file=config.local_data_files,
                unzip_dir=config.unzip_dir
          )
          return data_ingestion_config

    

In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers= request.urlretrieve(
                url= self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download with the following info: \n{headers}")
        else:
            logger.info(f"file already exists of size : {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-07-10 22:45:39,969:INFO:common:yaml file: config\config.yaml loaded successfully]
[2025-07-10 22:45:39,973:INFO:common:yaml file: params.yaml loaded successfully]
[2025-07-10 22:45:39,973:INFO:common:created directory at: artifacts]
[2025-07-10 22:45:39,973:INFO:common:created directory at: artifacts/data_ingestion]
[2025-07-10 22:45:55,616:INFO:2132865262:artifacts/data_ingestion/data.zip download with the following info: 
Connection: close
Content-Length: 7925852
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "fcb9f9eb37a5b0602dcfbf341ec336ddc9adf07e239a60bc6f8ca645cd40491c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 58FE:3A9035:132FD1:190394:686FFC46
Accept-Ranges: bytes
Date: Thu, 10 Jul 2025 17:45:44 GMT
Via: 1.1 varnish
X-Served-By: cache-fjr990020-FJR
X-Cache: MISS
X-Cache